In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from PIL import Image
import io
import datetime
import json
import re


In [2]:
dataset_name = "dataset_3"

In [3]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
timestamp

&#39;20201002-143122&#39;

In [836]:
dataset_dir = "../model/data/{}".format(dataset_name)

os.makedirs(dataset_dir)
os.makedirs(dataset_dir + "/images")

In [837]:
dataset_info = {
    "item_name": '앱솔랩스 브로드세이버',
    "server": '크로아',
    "timestamp": timestamp
}
with open("{}/{}_info.json".format(dataset_dir, dataset_name), "w", encoding='utf-16') as j:
    json.dump(dataset_info, j, ensure_ascii=False, indent=4)

In [838]:
# additional_option == 추옵 (무기류)
is_additional_option = True

In [839]:
url = "https://maple.market/items/{}/{}".format(dataset_info["item_name"].replace(" ", ""), dataset_info["server"])
url

&#39;https://maple.market/items/앱솔랩스브로드세이버/크로아&#39;

In [840]:
req = requests.get(url)
html = req.text

In [841]:
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table')
table_head = table.select('thead')[0]
table_body = table.select('tbody')[0]
items = table_body.find_all("tr")

In [842]:
columns = ["price", "option", "additional", "star_force", "upgrade", "time_remained"]
columns

[&#39;price&#39;, &#39;option&#39;, &#39;additional&#39;, &#39;star_force&#39;, &#39;upgrade&#39;, &#39;time_remained&#39;]

In [843]:
def item_parser(item, is_additional_option=False):      
    # additional_option == 추옵 (무기류)
    if is_additional_option:
        mapping = [1, 3, 4, 5, 7]
    else:
        mapping = [1, 2, 3, 4, 6]

    data = []
    item_contents = item.find_all("td")
    
    #num = int(item_contents[0].text.strip().replace('\n',''))
    #item_name = item_contents[1].select("span")[0].text.strip()

    # 1, 2
    item_name_span = item_contents[mapping[0]].select("span")[0]
    star_force = 0
    upgrade = 0
    item_name_raw = item_name_span.text.strip()

    if item_name_span.find("div") != None:
        star_force = int(item_name_raw[-1])
        item_name_raw = item_name_raw[:-1].strip()

    m = re.compile("\+(\d*)")

    if m.search(item_name_raw) != None:
        upgrade = int(m.search(item_name_raw).group(1))

    # 3
    option = item_contents[mapping[1]].text.strip()

    # 4
    additional = item_contents[mapping[2]].text.strip()

    # 5 
    price = int(item_contents[mapping[3]].text.strip().split("\n")[0].replace(',',''))

    # 6
    time_remained = item_contents[mapping[4]].text.strip()

    data.append(price)
    data.append(option)
    data.append(additional)
    data.append(star_force)
    data.append(upgrade)
    data.append(time_remained)

    return data

In [844]:
def item_image_parser(item):
    item_image_url = item.find_all("td")[1].select("div")[0].attrs["data-tooltip-image-url"]
    item_image_content = requests.get(item_image_url)
    assert(item_image_content.status_code == 200)

    stream = io.BytesIO(item_image_content.content)
    item_image = Image.open(stream)

    return item_image

In [845]:
current_data = []
current_images = []

for i, item in enumerate(items):
    d = item_parser(item, is_additional_option=is_additional_option)
    current_data.append(d)
    img = item_image_parser(item)
    current_images.append(img)
    print(i)

assert(len(current_data) == len(current_images))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183


In [846]:
current_df = pd.DataFrame(current_data, columns=columns, index=None)
current_df

,price,option,additional,star_force,upgrade,time_remained
0,99999999,-,-,0,0,22시간 30분
1,99999999,-,-,0,0,22시간 57분
2,109999998,-,-,0,0,20시간 50분
3,109999999,-,-,0,0,13시간 52분
4,110000000,-,-,0,0,05시간 29분
...,...,...,...,...,...,...
179,13999999999,레전,레전,2,9,23시간 28분
180,14999999999,레전,유닉,2,9,14시간 50분
181,19999999999,레전,레전,2,9,04시간 51분
182,20299999999,레전,유닉,2,9,05시간 40분


In [847]:
current_df.to_csv("{}/{}_current.csv".format(dataset_dir, dataset_name), index=None, encoding='utf-8')

In [848]:
for i, img in enumerate(current_images):
    img.save("{}/images/{}.png".format(dataset_dir, i))